In [5]:
# import pandas as pd

# for coin in ['BTCUSDT', 'ETHUSDT', 'SOLUSDT', 'DOGEUSDT', 'ADAUSDT', 'BNBUSDT', 'LINKUSDT', 'TRXUSDT', 'XRPUSDT', 'USDCUSDT']:
#     cols = [
#         "open_time", "open", "high", "low", "close", "volume",
#         "close_time", "quote_volume", "num_trades",
#         "tbb_volume", "tbq_volume", "ignore"
#     ]

#     df = pd.read_csv(f"data/{coin}.csv", header=None, names=cols)

#     # Convert to proper dtypes
#     df["open_time"] = df["open_time"].astype("int64")
#     df["close_time"] = df["close_time"].astype("int64")

#     numeric_cols = [
#         "open", "high", "low", "close", "volume",
#         "quote_volume", "num_trades", "tbb_volume", "tbq_volume"
#     ]
#     df[numeric_cols] = df[numeric_cols].astype(float)

#     # ---- 2. Use open_time (ms) as datetime index for resampling ----
#     df["dt"] = pd.to_datetime(df["open_time"], unit="ms")
#     df = df.set_index("dt")

#     # ---- 3. Resample to hourly candles ----
#     # Rules:
#     #   open  = first open of the hour
#     #   high  = max high
#     #   low   = min low
#     #   close = last close
#     #   volume, quote_volume, trades, taker buy volumes = sum
#     #   ignore = last (usually 0 anyway)
#     agg_dict = {
#         "open": "first",
#         "high": "max",
#         "low": "min",
#         "close": "last",
#         "volume": "sum",
#         "quote_volume": "sum",
#         "num_trades": "sum",
#         "tbb_volume": "sum",
#         "tbq_volume": "sum",
#         "ignore": "last",
#     }

#     hourly = df.resample("D").agg(agg_dict)

#     # Drop empty hours (no trades)
#     hourly = hourly.dropna(subset=["open", "high", "low", "close"])

#     # ---- 4. Rebuild open_time and close_time in ms (Binance style) ----
#     # Index is start-of-hour; convert to ms
#     hourly["open_time"] = (hourly.index.view("int64") // 10**6).astype("int64")

#     # close_time = end of the hour minus 1 ms
#     ONE_HOUR_MS = 60 * 60 * 1000
#     hourly["close_time"] = hourly["open_time"] + ONE_HOUR_MS - 1

#     # ---- 5. Reorder columns to match original exactly ----
#     hourly_out = hourly[
#         [
#             "open_time",
#             "open",
#             "high",
#             "low",
#             "close",
#             "volume",
#             "close_time",
#             "quote_volume",
#             "num_trades",
#             "tbb_volume",
#             "tbq_volume",
#             "ignore",
#         ]
#     ]

#     # ---- 6. Write back out: same style (no header, comma-separated) ----
#     hourly_out.to_csv(
#         f"{coin}.csv",
#         header=False,
#         index=False,
#         float_format="%.8f"  # adjust precision if you like
#     )


In [6]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.collections import LineCollection
from sklearn.manifold import MDS
import networkx as nx
from networkx.algorithms.community import greedy_modularity_communities, modularity


input_dir = 'data/'
symbols = ['USDCUSDT', 'BTCUSDT', 'ETHUSDT', 'SOLUSDT', 'BNBUSDT', 'XRPUSDT', 'DOGEUSDT', 'AVAXUSDT', 'ZECUSDT', 'ALPINEUSDT', 'ADAUSDT', 'TRXUSDT', 'LINKUSDT', 'LTCUSDT', 'APTUSDT', 'CAKEUSDT', 'NEARUSDT', 'ARBUSDT', 'AAVEUSDT', 'HBARUSDT', 'SUPERUSDT', 'XLMUSDT', 'DOTUSDT', 'PAXGUSDT', 'ZENUSDT', 'UNIUSDT', 'LDOUSDT', 'CRVUSDT', 'DASHUSDT', 'EURUSDT', 'BCHUSDT', 'SNXUSDT', 'FETUSDT', 'XVGUSDT', 'FILUSDT', 'SHIBUSDT', 'OPUSDT', 'TWTUSDT', 'ICPUSDT', 'C98USDT', 'ATOMUSDT', 'INJUSDT', 'ARUSDT', 'ETCUSDT', 'GALAUSDT', 'HOOKUSDT', 'SUNUSDT', 'ALGOUSDT', 'OMUSDT', 'RUNEUSDT', 'IMXUSDT', 'OGUSDT', 'SANTOSUSDT', 'VETUSDT', 'CELOUSDT', 'ATMUSDT', 'RAYUSDT', 'LAZIOUSDT', 'CFXUSDT', 'DEXEUSDT', 'SANDUSDT', 'NMRUSDT', 'CHZUSDT', 'RSRUSDT', 'SKLUSDT', 'YGGUSDT', 'DYDXUSDT', 'ENSUSDT', 'APEUSDT', 'MANAUSDT', 'QNTUSDT', 'FXSUSDT', 'SUSHIUSDT', 'GRTUSDT', 'TRBUSDT', 'NEOUSDT', 'ILVUSDT', 'FIDAUSDT', 'STGUSDT', '1INCHUSDT', 'WOOUSDT', 'THETAUSDT', 'LPTUSDT', 'SSVUSDT', 'PEOPLEUSDT', 'JUVUSDT', 'IOTAUSDT', 'RLCUSDT', 'FTTUSDT', 'KDAUSDT', 'STXUSDT', 'ALICEUSDT', 'PROMUSDT', 'MINAUSDT', 'EGLDUSDT', 'GMXUSDT', 'JASMYUSDT', 'IDEXUSDT', 'SCRTUSDT', 'ROSEUSDT', 'RAREUSDT', 'YFIUSDT', 'ASTRUSDT', 'API3USDT', 'LUNCUSDT', 'AXSUSDT', 'ASRUSDT', 'KSMUSDT', 'KAVAUSDT', 'CVXUSDT', 'COMPUSDT', 'MBLUSDT', 'IDUSDT', 'FUNUSDT', 'LRCUSDT', 'SFPUSDT', 'CITYUSDT', 'ACHUSDT', 'GMTUSDT', 'AUCTIONUSDT', 'XVSUSDT', 'ONEUSDT', 'IOTXUSDT', 'HFTUSDT', 'FISUSDT', 'PORTOUSDT', 'GNOUSDT', 'DCRUSDT', 'OSMOUSDT', 'XTZUSDT', 'RVNUSDT', 'RADUSDT', 'PHAUSDT', 'SYNUSDT', 'LUNAUSDT', 'PYRUSDT', 'NEXOUSDT', 'ACMUSDT', 'VOXELUSDT', 'BTTCUSDT', 'CTSIUSDT', 'USTCUSDT', 'BARUSDT', 'OGNUSDT', 'ZRXUSDT', 'CKBUSDT', 'ZILUSDT', 'MAGICUSDT', 'ONTUSDT', 'LQTYUSDT', 'JOEUSDT', 'QTUMUSDT', 'ONGUSDT', 'DEGOUSDT', 'MDTUSDT', 'JSTUSDT', 'GLMRUSDT', 'ENJUSDT', 'MASKUSDT', 'QUICKUSDT', 'FLOWUSDT', 'PERPUSDT', 'COTIUSDT', 'CHRUSDT', 'QIUSDT', 'PHBUSDT', 'PUNDIXUSDT', 'HIGHUSDT', 'PSGUSDT', 'MBOXUSDT', 'DUSKUSDT', 'UMAUSDT', 'MOVRUSDT', 'BELUSDT', 'RPLUSDT', 'GTCUSDT', 'ACAUSDT', 'BANDUSDT', 'AUDIOUSDT', 'TRUUSDT', 'USDPUSDT', 'OXTUSDT', 'UTKUSDT', 'AMPUSDT', 'CHESSUSDT', 'AVAUSDT', 'CTKUSDT', 'ARPAUSDT', 'COSUSDT', 'DGBUSDT', 'FLUXUSDT', 'XECUSDT', 'HOTUSDT', 'TLMUSDT', 'SXPUSDT', 'ANKRUSDT', 'AGLDUSDT', 'SLPUSDT', 'DODOUSDT', 'SPELLUSDT', 'BATUSDT', 'STORJUSDT', 'KNCUSDT', 'STRAXUSDT', 'IOSTUSDT', 'POLYXUSDT', 'NKNUSDT', 'GLMUSDT', 'TFUELUSDT', 'STEEMUSDT', 'TUSDT', 'GASUSDT', 'DIAUSDT', 'BICOUSDT', 'DENTUSDT', 'ADXUSDT', 'MLNUSDT', 'SCUSDT', 'LSKUSDT', 'ICXUSDT', 'VTHOUSDT', 'CELRUSDT', 'TKOUSDT', 'QKCUSDT', 'SYSUSDT', 'WAXPUSDT', 'HIVEUSDT', 'ATAUSDT', 'XNOUSDT', 'TUSDUSDT', 'FIOUSDT', 'FARMUSDT', 'WINUSDT', 'ALCXUSDT', 'PONDUSDT', 'FLMUSDT', 'CVCUSDT', 'FORTHUSDT', 'DFUSDT', 'DATAUSDT', 'BIFIUSDT', 'REIUSDT', 'BNTUSDT', 'MTLUSDT', 'GNSUSDT', 'POWRUSDT', 'ARDRUSDT', 'WANUSDT', 'RIFUSDT', 'GHSTUSDT', 'REQUSDT']
candlestick_names = [
    "Open time",
    "Open",
    "High",
    "Low",
    "Close",
    "Volume",
    "Close time",
    "Quote asset volume",
    "Number of trades",
    "Taker buy base asset volume",
    "Taker buy quote asset volume",
    "Ignore"
]
def load_series(symbol, col_indices):
    time_serieses = []
    col_names = [f'{symbol}_{candlestick_names[i]}' for i in col_indices]
    df = pd.read_csv(
            os.path.join(input_dir, f"{symbol}.csv"),
            usecols=[0, *col_indices],
            header = None,
            names=["time", *col_names]
        )
    df.loc[df["time"] >= 1e15, "time"] = df["time"] // 1000
    df["time"] = pd.to_datetime(df["time"], unit="ms")
    df = df[df["time"] >= pd.Timestamp('2023-03-24 14:00:00')]
    for name in col_names:
        s = pd.Series(
            df[name].values,
            index = df["time"],
            name = name
        )
        time_serieses.append(s)
    return time_serieses
def load_dataframes(symbols, col_indices):
    dfs = [[] for _ in col_indices]
    for symbol in symbols:
        time_serieses = load_series(symbol, col_indices)
        for i, time_series in enumerate(time_serieses):
            dfs[i].append(time_series)
    for i, col_index in enumerate(col_indices):
        dfs[i] = pd.concat(dfs[i], axis=1)
    return dfs

price_df_na, = load_dataframes(['BTCUSDT', 'ETHUSDT', 'SOLUSDT', 'DOGEUSDT', 'ADAUSDT', 'BNBUSDT', 'LINKUSDT', 'TRXUSDT', 'XRPUSDT', 'USDCUSDT'], [1])

price_df_na.to_csv('timeseries.csv')